# Definición de la neurona
Se define la estructura de la neurona, la cual contendrá toda la información necesaria para que pueda realizar las operaciones que requiere.

In [61]:
import random

class Neuron:
  def __init__(self, function, value=0.0, id=None):
    self.value = value
    self.id = id
    self.funct = function
    self.der_funct = lambda F_x: F_x * (1 - F_x)
    self.next_layer = []
    self.prev_layer = []
    self.curr_weights = []
    self.bias = 1


  def add_layer(self, nodes_layer=None):
    self.prev_layer = nodes_layer
    
    for node in nodes_layer:
      node.next_layer.append(self)
      
    self.curr_weights = [random.random() for n in nodes_layer]


  def calculate_value(self, apply_function=True):
    sum = 0.0
    for (index, node) in enumerate(self.prev_layer):
      sum += node.value * self.curr_weights[index]

    self.value = sum + self.bias

    if apply_function:
      self.value = self.funct(self.value)

    return self.value

## Probando la neurona
Se realizan algunas pruebas para verificar que todas las funciones las realiza de una forma correcta.
  Se crea una neurona que use la función sigmoide como función de activación, luego se le agrega una capa a partir de los valores que deberian tener los nodos. Por ultimo se calcula el valor activado de la neurona.

In [64]:
import math

def sigmoide(x):
  return 1 / (1 + math.exp(-x))

values = [0.34, 0.99, 0.03]
nodes = [Neuron(sigmoide, v) for v in values]

n = Neuron(sigmoide)
n.add_layer(nodes)

n.calculate_value()

0.8098635551430515

# Definición de la red neuronal
Se defina la estructura de la red neuronal, la cual tendra las funciones básicas para que pueda operar.

In [85]:
class NeuronalNetwork:
  def __init__(self, cant_input, cant_output, function):
    self.input_layer = [Neuron(function, id=(0, i + 1)) for i in range(cant_input)]
    self.output_layer = [Neuron(function, id=(-1, i + 1)) for i in range(cant_output)]
    self.hidden_layers = []

  def is_empty(self):
    return len(self.hidden_layers) == 0

  def get_function(self):
    return self.input_layer[0].funct

  def add_layers(self, layers):
    # Se obtiene la malla
    self.hidden_layers = self._make_connections(layers)

   # Se conecta el principio de la malla con la capa de entrada
    for node in self.hidden_layers[0]:
      node.add_layer(nodes_layer=self.input_layer)

    # Se conecta la capa de salida con el final de la malla
    for node in self.output_layer:
      node.add_layer(nodes_layer=self.hidden_layers[-1])


  def fit(self, X, y, learning_rate=0.1):
    # Por cada vector del conjunto de inputs, se ajusta el modelo
    for x in X:
      self.predict(x)
      deltas = self._calculate_deltas()
      self._update_weights(deltas, learning_rate)

  
  def predict(self, input):
    # Se copian los valores del input en los nodos de la capa de entrada
    for (index, value) in enumerate(input):
      self.input_layer[index].value = value

    # Se calculan los valores de las capas ocultas
    for layer in self.hidden_layers:
      for node in layer:
        node.calculate_value()
        print(f'{node.id}: {node.value}')

    # Se calcula la salida sin aplicar la funcion de activacion al valor
    for node in self.output_layer:
      node.calculate_value()

    return [node.value for node in self.output_layer]


  def _make_connections(self, layers):
    '''
    Realiza las conexiones entre las capas ingresadas, y retorna una malla de 
    la red neuronal (sin entrada ni salida)
    '''
    nodes_mesh = self._create_mesh(layers)
    length = len(layers)

    for i in range(1, length):
      for node in nodes_mesh[i]:
        node.add_layer(nodes_layer=nodes_mesh[i - 1])

    return nodes_mesh


  def _create_mesh(self, layers):
    '''
    Crea una malla de capas, pero sin conecciones entre ellas
    '''
    nodes_mesh = []
    function = self.get_function()

    for current_layer in layers:
      new_nodes_layer = [Neuron(function, id=(current_layer, j + 1)) for j in range(current_layer)] # for debug
      #new_nodes_layer = [current_layer for j in range(current_layer)]  # for debug
      #new_nodes_layer = [Neuron(function) for j in range(current_layer)]
      nodes_mesh.append(new_nodes_layer)

    return nodes_mesh


  def _update_weights(self, deltas, learning_rate):
    # Se actualizan los pesos de la primera capa oculta
    self._update_layer_weight(self.input_layer, 0, deltas, learning_rate)

    # Se actualizan los pesos de la segunda capa oculta
    for (i, layer) in enumerate(self.hidden_layers):
      self._update_layer_weight(layer, i + 1, deltas, learning_rate)


  def _update_layer_weight(self, layer, i_delta, deltas, learning_rate):
    for (i, node) in enumerate(layer):
      for (j, next) in enumerate(node.next_layer):
        new_weight = -learning_rate * deltas[i_delta][j] * next.value
        next.curr_weights[i] = new_weight


  def _calculate_deltas(self):
    deltas = self._create_delta_set()
    
    # Se calculan los deltas de la capa de salida
    for (index, node) in enumerate(self.output_layer):
      deltas[-1][index] = (node.value - y[index]) * node.der_funct(node.value)

    length_hidden = len(self.hidden_layers)

    # Se calculan los deltas de las capas ocultas
    for i in range(length_hidden):
      i_lay = length_hidden - i - 1
      layer = self.hidden_layers[i_lay]

      for (j, node) in enumerate(layer):
        sum = 0.0

        for (k, next) in enumerate(node.next_layer):
          sum += deltas[i_lay + 1][k] * next.curr_weights[j]

        deltas[i_lay][j] = sum * node.der_funct(node.value)

    return deltas

  def _create_delta_set(self):
    deltas_hidden = [[0.0 for v in layer] for layer in self.hidden_layers]
    deltas_output = [0.0 for v in self.output_layer]

    return [*deltas_hidden, deltas_output]

In [79]:
layers = [2, 1, 3]

nn = NeuronalNetwork(3, 2, sigmoide)
nn.add_layers(layers)

In [84]:
X = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
y = [1, 2, 3]

nn.fit(X, y)

(2, 1): 0.7310585786302926
(2, 2): 0.7310585786302926
(1, 1): 0.7310585815295213
(3, 1): 0.7310625108504119
(3, 2): 0.7310625108786869
(3, 3): 0.7310625113053609


In [83]:
x = [3, 4.7, 7.8]
print(nn.predict(x))

(2, 1): 0.731058578630748
(2, 2): 0.731058578630748
(1, 1): 0.7310585815295213
(3, 1): 0.7310625108504119
(3, 2): 0.7310625108786869
(3, 3): 0.7310625113053609
[0.7327209211229652, 0.7388049389137314]


In [47]:
# For debugging
def show_mesh(mesh):
  for layer in mesh:
    for node in layer:
      for (index, neight) in enumerate(node.prev_layer):
        print(f'{node.id} -> {neight.id}  w: {node.curr_weights[index]} v:{node.value}')

In [48]:
# For debugging
def show_mesh_reverted(mesh):
  for layer in mesh:
    for (layer_index, node) in enumerate(layer):
      for (index, next_node) in enumerate(node.next_layer):
        print(f'{node.id} -> {next_node.id} w:{next_node.curr_weights[layer_index]}')

In [ ]:
d = 0.739304579579997 * 0.3 + 0.009 + 0.6535563064237191 * 0.7 + 0.12 + 0.8843939651086795 * 0.1 + 0.023
sigmoide(d)

0.7149850880113883

In [50]:
mesh = [*nn.hidden_layers]
mesh.insert(0, nn.input_layer)
mesh.append(nn.output_layer)

In [51]:
print('cantidad de capas totales:', len(mesh))

show_mesh(mesh)

cantidad de capas totales: 5
(2, 1) -> (0, 1)  w: 0.00012690555604301777 v:0.6252674803279497
(2, 1) -> (0, 2)  w: 0.00012690555604301777 v:0.6252674803279497
(2, 1) -> (0, 3)  w: 0.00012690555604301777 v:0.6252674803279497
(2, 2) -> (0, 1)  w: 0.00013062974440666757 v:0.5453809587378066
(2, 2) -> (0, 2)  w: 0.00013062974440666757 v:0.5453809587378066
(2, 2) -> (0, 3)  w: 0.00013062974440666757 v:0.5453809587378066
(1, 1) -> (2, 1)  w: 0.0009012677211750377 v:0.5077631430137003
(1, 1) -> (2, 2)  w: 0.0009012677211750377 v:0.5077631430137003
(3, 1) -> (1, 1)  w: 0.0014601867925911692 v:0.51018394863359
(3, 2) -> (1, 1)  w: 0.0030535164873163674 v:0.5128847625239672
(3, 3) -> (1, 1)  w: 0.0004452164447185362 v:0.5075559408656884
(-1, 1) -> (3, 1)  w: 0.004166870398071568 v:0.5514121299252297
(-1, 1) -> (3, 2)  w: 0.004166870398071568 v:0.5514121299252297
(-1, 1) -> (3, 3)  w: 0.004166870398071568 v:0.5514121299252297
(-1, 2) -> (3, 1)  w: 0.01781487764902003 v:0.5446973336938459
(-1, 2) 

In [52]:
show_mesh_reverted(mesh)

(0, 1) -> (2, 1) w:0.00012690555604301777
(0, 1) -> (2, 2) w:0.00013062974440666757
(0, 2) -> (2, 1) w:0.00012690555604301777
(0, 2) -> (2, 2) w:0.00013062974440666757
(0, 3) -> (2, 1) w:0.00012690555604301777
(0, 3) -> (2, 2) w:0.00013062974440666757
(2, 1) -> (1, 1) w:0.0009012677211750377
(2, 2) -> (1, 1) w:0.0009012677211750377
(1, 1) -> (3, 1) w:0.0014601867925911692
(1, 1) -> (3, 2) w:0.0030535164873163674
(1, 1) -> (3, 3) w:0.0004452164447185362
(3, 1) -> (-1, 1) w:0.004166870398071568
(3, 1) -> (-1, 2) w:0.01781487764902003
(3, 2) -> (-1, 1) w:0.004166870398071568
(3, 2) -> (-1, 2) w:0.01781487764902003
(3, 3) -> (-1, 1) w:0.004166870398071568
(3, 3) -> (-1, 2) w:0.01781487764902003
